# Isochrones with Python and APIS

In [87]:
# We'll use this to output some of our intermediate data structures in a somewhat standard format
import json

# We'll use this to break our address into street, city, state pieces for us
import usaddress

# Lots of meetings are at the Cook County Building.  This is one example from the data.
location_name = "Cook County Building, Board Room, 118 North Clark Street, Chicago, Illinois"

# Many geocoders, especially free ones need us to break the address into pieces.
# Let's use the awesome usaddress package to do this for us
parsed_location_name, detected_type = usaddress.tag(location_name)

print(json.dumps(parsed_location_name, indent=4))

{
    "BuildingName": "Cook County Building, Board Room",
    "AddressNumber": "118",
    "StreetNamePreDirectional": "North",
    "StreetName": "Clark",
    "StreetNamePostType": "Street",
    "PlaceName": "Chicago",
    "StateName": "Illinois"
}


In [88]:
# Let's merge these very atomic fields into ones that are closer to those
# accepted by many geocoding APIs
street_address = ' '.join([
    parsed_location_name['AddressNumber'],
    parsed_location_name['StreetNamePreDirectional'],
    parsed_location_name['StreetName'],
    parsed_location_name['StreetNamePostType']
])
address = {
    'street_address': street_address,
    'city': parsed_location_name['PlaceName'],
    'state': parsed_location_name['StateName']
}

print(json.dumps(address, indent=4))

{
    "city": "Chicago",
    "state": "Illinois",
    "street_address": "118 North Clark Street"
}


In [89]:
import csv
import io

# We'll use this to simplify making our HTTP requests to the geocoding APIs
import requests

# Let's merge our parsed address into fields that are

# We'll use the Texas A&M Geoservices API because it's free (for few requests)
# and has pretty liberal terms of service.  It does however require an API
# key.
TAMU_GEOSERVICES_API_KEY = os.environ.get('TAMU_GEOSERVICES_API_KEY')

# API parameters, passed via the URL
# That is:
# https://geoservices.tamu.edu/Services/Geocode/WebService/GeocoderWebServiceHttpNonParsed_V04_01.aspx?version=4.01&streetAddres=118+North+Clark+Street...
api_params = {
    'apiKey': TAMU_GEOSERVICES_API_KEY,
    'version': '4.01',
    'streetAddress': address['street_address'],
    'city': address['city'],
    'state': address['state'],
    'includeHeader': 'true',
}

api_url = 'https://geoservices.tamu.edu/Services/Geocode/WebService/GeocoderWebServiceHttpNonParsed_V04_01.aspx'
r = requests.get(api_url, params=api_params)


# The TAMU geocoder returns the data as CSV.  We need to parse this out

geocoded = None
inf = io.StringIO(r.text)
reader = csv.DictReader(inf)

for row in reader:
    geocoded = row
    # Only read one row
    break
    
print(json.dumps(geocoded, indent=2))

{
  "FeatureMatchingGeographyType": "StreetSegment",
  "": "",
  "Version": "4.1",
  "TransactionId": "be5319e6-1f88-4f8a-97b0-3285b42d138a",
  "Latitude": "41.8834456872408",
  "TimeTaken": "0.0430043",
  "NAACCRGISCoordinateQualityCode": "03",
  "RegionSize": "3174.85431915501",
  "MatchScore": "96.0059171597633",
  "Longitude": "-87.631003848859",
  "RegionSizeUnits": "Meters",
  "MatchType": "Relaxed",
  "MatchedLocationType": "LOCATION_TYPE_STREET_ADDRESS",
  "QueryStatusCodeValue": "200",
  "FeatureMatchingResultCount": "1",
  "NAACCRGISCoordinateQualityName": "StreetSegmentInterpolation",
  "FeatureMatchingResultType": "Success"
}


In [90]:
# Our longitude and latitude are available as keys of the parsed dictionary
print("[{0}, {1}]".format(geocoded['Longitude'], geocoded['Latitude']))

[-87.631003848859, 41.8834456872408]


In [91]:
from datetime import datetime

from pytz import timezone

start_time = "March 8, 2017 5:00pm"

parsed_start_time = datetime.strptime(start_time, "%B %d, %Y %I:%M%p")

# Convert to Chicago's time zone
central = timezone('US/Central')
parsed_start_time = central.localize(parsed_start_time)

print(parsed_start_time.isoformat())

2017-03-08T17:00:00-06:00


## Get Isochrones from the MapZen Isochrone service

In [92]:
# http://colorbrewer2.org/?type=sequential&scheme=BuGn&n=5
COLORS_BLUE_GREEN = [
    '#edf8fb',
    '#b2e2e2',
    '#66c2a4',
    '#2ca25f',
    '#006d2c',
]

In [93]:
import os

import requests

# Read our API key from an environment variable.
# This is a good way to avoid hard-coding credentials like API keys in our code.
MAPZEN_API_KEY = os.environ.get('MAPZEN_API_KEY')

# See https://mapzen.com/documentation/mobility/isochrone/api-reference/ for the API reference
# These are API parameters we'll use for all of the transportation types we're trying
base_api_params = {
    'api_key': MAPZEN_API_KEY,
    # Location and costing (transportation type)
    'json': {
        'locations': [
            {
                'lat': geocoded['Latitude'],
                'lon': geocoded['Longitude'],
            },
        ], 
        # Contours
        # You can have a maximum of 4
        'contours': [
            {
                'time': 30,
                'color': COLORS_BLUE_GREEN[0].lstrip('#'),
            },
            {
                'time': 60,
                'color': COLORS_BLUE_GREEN[2].lstrip('#'),
            },
            {
                'time': 90,
                'color': COLORS_BLUE_GREEN[4].lstrip('#'),
            },
        ],
        # Return GeoJSON polygons instead of linestrings.
        # Linestrings are smaller and faster to render, but we use polygons for colored shading.
        'polygons': True,
        # Distance in meters used as an input to simplify the polygons.
        # A higher number will result in more simplifications.
        # I'm doing this here, because without some simplification, the GeoJSON is too long
        # to pass as a URL parameter to geojson.io, which we're using to display our maps.
        'generalize': 300,
    },
}

### Multimodal (transit + walking)

[Example](http://bl.ocks.org/d/be742d6ba8cfbb1f19683efd3557ee29)

In [94]:
from copy import deepcopy
import datetime
import json

api_params = deepcopy(base_api_params)

# Calculate a start time 1 hour before the meeting time.  This is hacky, but the API doesn't support an arrival time
# yet.
departure_time = parsed_start_time - datetime.timedelta(hours=1)

# Specify transit/walking
api_params['json']['costing'] = 'multimodal'
api_params['json']['date_time'] = {
    # `1` means specified departure time.  Ideally we would use `2`, the arrival time, but the API docs say this
    # isn't supported yet.
    'type': 1,
    'value': departure_time.replace(tzinfo=None).strftime("%Y-%m-%dT%H:%M"),
}

# Convert `json` parameter to JSON
api_params['json'] = json.dumps(api_params['json'])

r = requests.get('https://matrix.mapzen.com/isochrone', params=api_params)

contour_geojson = json.dumps(r.json())

In [95]:
import geojsonio

# Embed a map in this notebook
# This should work, but doesn't.  Maybe it's just a problem on Linux
#geojsonio.embed(contour_geojson)

geojsonio.display(contour_geojson)

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22properties%22%3A%20%7B%22fillOpacity%22%3A%200.33%2C%20%22color%22%3A%20%22%23006d2c%22%2C%20%22fillColor%22%3A%20%22%23006d2c%22%2C%20%22opacity%22%3A%200.33%2C%20%22fill-opacity%22%3A%200.33%2C%20%22fill%22%3A%20%22%23006d2c%22%2C%20%22contour%22%3A%2090%7D%2C%20%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22coordinates%22%3A%20%5B%5B%5B-87.823006%2C%2042.227806%5D%2C%20%5B-87.826485%2C%2042.218956%5D%2C%20%5B-87.817093%2C%2042.210449%5D%2C%20%5B-87.820709%2C%2042.204449%5D%2C%20%5B-87.816307%2C%2042.194752%5D%2C%20%5B-87.820198%2C%2042.192249%5D%2C%20%5B-87.83065%2C%2042.198448%5D%2C%20%5B-87.822716%2C%2042.183731%5D%2C%20%5B-87.825531%2C%2042.177914%5D%2C%20%5B-87.819771%2C%2042.174671%5D%2C%20%5B-87.810997%2C%2042.177567%5D%2C%20%5B-87.801811%2C%2042.168255%5D%2C%20%5B-87.819107%2C%2042.170559%5D%2C%20%5B-87.826004%2C%2042.154205%5D%2C%20%5B-87.

## Bicycling

[Example](http://bl.ocks.org/d/6539fcf5a3ed64d052a01a17ac035a68)

In [96]:
from copy import deepcopy
import json

api_params = deepcopy(base_api_params)

# Specify bicycling costing methods
# There are a lot more options, just use the defaults for now
# See https://mapzen.com/documentation/mobility/turn-by-turn/api-reference/#bicycle-costing-options
api_params['json']['costing'] = 'bicycle'

# Convert `json` parameter to JSON
api_params['json'] = json.dumps(api_params['json'])

r = requests.get('https://matrix.mapzen.com/isochrone', params=api_params)

contour_geojson = json.dumps(r.json())

In [97]:
geojsonio.display(contour_geojson)

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22properties%22%3A%20%7B%22fillOpacity%22%3A%200.33%2C%20%22color%22%3A%20%22%23006d2c%22%2C%20%22fillColor%22%3A%20%22%23006d2c%22%2C%20%22opacity%22%3A%200.33%2C%20%22fill-opacity%22%3A%200.33%2C%20%22fill%22%3A%20%22%23006d2c%22%2C%20%22contour%22%3A%2090%7D%2C%20%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22coordinates%22%3A%20%5B%5B%5B-87.683006%2C%2042.078148%5D%2C%20%5B-87.701012%2C%2042.070118%5D%2C%20%5B-87.705009%2C%2042.051373%5D%2C%20%5B-87.713005%2C%2042.053883%5D%2C%20%5B-87.733818%2C%2042.037449%5D%2C%20%5B-87.745193%2C%2042.037445%5D%2C%20%5B-87.737747%2C%2042.040184%5D%2C%20%5B-87.739006%2C%2042.052952%5D%2C%20%5B-87.761009%2C%2042.05254%5D%2C%20%5B-87.763008%2C%2042.043095%5D%2C%20%5B-87.771004%2C%2042.044697%5D%2C%20%5B-87.783005%2C%2042.035263%5D%2C%20%5B-87.788239%2C%2042.038673%5D%2C%20%5B-87.789001%2C%2042.029243%5D%2C%20%5B-87.

## Get Isocrones from the HERE Routing API

[Example](http://bl.ocks.org/d/6838e193d9f72a7f14e08ab42fefa7ad)

In [107]:
import os

import requests

# Let's get an isochrone using the HERE Routing API Isoline endpoint

HERE_APP_ID = os.environ.get('HERE_APP_ID')
HERE_APP_CODE = os.environ.get('HERE_APP_CODE')

api_params = {
    'app_id': HERE_APP_ID,
    'app_code': HERE_APP_CODE,
    'mode': "fastest;car;traffic:enabled",
    'destination': "geo!{lat},{lng}".format(lat=geocoded['Latitude'], lng=geocoded['Longitude']),
    'arrival': parsed_start_time.isoformat(),
    # 1 hour
    'range': 1 * 60 * 60,
    'rangetype': 'time'
}

r = requests.get("https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json", params=api_params)

isoline_response = r.json()['response']

In [108]:
# Convert the isoline API response to GeoJSON
import json

isoline_geojson = {
    'type': 'FeatureCollection',
    'features': [],
}

isoline_geojson['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [
            isoline_response['center']['longitude'],
            isoline_response['center']['latitude']
        ],
    },
    'properties': {},
})

isoline_geojson['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Polygon',
        'coordinates': [
            [
                [float(c) for c in x.split(',')[::-1]]
                for x
                in isoline_response['isoline'][0]['component'][0]['shape']
            ],
        ],
    },
    'properties': {},
})


In [109]:
# And display it using geojson.io

import geojsonio
geojsonio.display(json.dumps(isoline_geojson))

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22properties%22%3A%20%7B%7D%2C%20%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22coordinates%22%3A%20%5B-87.6310039%2C%2041.8834456%5D%2C%20%22type%22%3A%20%22Point%22%7D%7D%2C%20%7B%22properties%22%3A%20%7B%7D%2C%20%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22coordinates%22%3A%20%5B%5B%5B-88.2408142%2C%2041.8139648%5D%2C%20%5B-88.1982422%2C%2041.8139648%5D%2C%20%5B-88.1900024%2C%2041.8167114%5D%2C%20%5B-88.1845093%2C%2041.8222046%5D%2C%20%5B-88.1762695%2C%2041.8249512%5D%2C%20%5B-88.1652832%2C%2041.8249512%5D%2C%20%5B-88.1542969%2C%2041.8276978%5D%2C%20%5B-88.1433105%2C%2041.8331909%5D%2C%20%5B-88.1323242%2C%2041.8359375%5D%2C%20%5B-88.1213379%2C%2041.8359375%5D%2C%20%5B-88.1130981%2C%2041.8386841%5D%2C%20%5B-88.1103516%2C%2041.8469238%5D%2C%20%5B-88.1103516%2C%2041.8579102%5D%2C%20%5B-88.107605%2C%2041.8661499%5D%2C%20%5B-88.0911255%2C

In [119]:
# Factor this example into functions so we can get multiple areas

import requests

def get_isoline(lat, lng, mode, rng, arrival, app_id, app_code):
    api_params = {
        'app_id': app_id,
        'app_code': app_code,
        'mode': mode,
        'destination': "geo!{lat},{lng}".format(lat=lat, lng=lng),
        'arrival': arrival.isoformat(),
        'range': rng,
        'rangetype': 'time'
    }

    r = requests.get("https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json", params=api_params)

    return r.json()['response']

def isoline_geojson_feature(isoline):
    return {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': [
                [
                    [float(c) for c in x.split(',')[::-1]]
                    for x
                    in isoline['isoline'][0]['component'][0]['shape']
                ],
            ],
        },
        'properties': {},
    }



In [124]:
ranges = [
    {
        'range': 1 * 60 * 60,
        'properties': {
            'fill': COLORS_BLUE_GREEN[0],
        },
    },
    {
        'range': 1 * 60 * 30,
        'properties': {
            'fill': COLORS_BLUE_GREEN[1],
        },
    },
]

isolines = {
    'type': 'FeatureCollection',
    'features': [],
}

isolines['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [
            float(geocoded['Longitude']),
            float(geocoded['Latitude']), 
        ],
    },
    'properties': {},
})

for rng in ranges:
    isoline = get_isoline(
        geocoded['Latitude'], 
        geocoded['Longitude'], 
        'fastest;car;traffic:enabled',
        rng['range'],
        parsed_start_time,
        HERE_APP_ID,
        HERE_APP_CODE
    )

    isoline_feature = isoline_geojson_feature(isoline)
    isoline_feature['properties'].update(**rng['properties'])
    isoline_feature['properties']['range'] = rng['range']
    isolines['features'].append(isoline_feature)

In [125]:
# And display it using geojson.io

import geojsonio

# TODO: Simplify GeoJSON because it's too long to pass to geojson.io via the URL
geojsonio.display(json.dumps(isolines))

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22properties%22%3A%20%7B%7D%2C%20%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22coordinates%22%3A%20%5B-87.631003848859%2C%2041.8834456872408%5D%2C%20%22type%22%3A%20%22Point%22%7D%7D%2C%20%7B%22properties%22%3A%20%7B%22range%22%3A%203600%2C%20%22fill%22%3A%20%22%23edf8fb%22%7D%2C%20%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22coordinates%22%3A%20%5B%5B%5B-88.2408142%2C%2041.8139648%5D%2C%20%5B-88.1982422%2C%2041.8139648%5D%2C%20%5B-88.1900024%2C%2041.8167114%5D%2C%20%5B-88.1845093%2C%2041.8222046%5D%2C%20%5B-88.1762695%2C%2041.8249512%5D%2C%20%5B-88.1652832%2C%2041.8249512%5D%2C%20%5B-88.1542969%2C%2041.8276978%5D%2C%20%5B-88.1433105%2C%2041.8331909%5D%2C%20%5B-88.1323242%2C%2041.8359375%5D%2C%20%5B-88.1213379%2C%2041.8359375%5D%2C%20%5B-88.1130981%2C%2041.8386841%5D%2C%20%5B-88.1103516%2C%2041.8469238%5D%2C%20%5B-88.1103516%2C%2041.85

In [123]:
print(json.dumps(isolines))

{"type": "FeatureCollection", "features": [{"properties": {}, "type": "Feature", "geometry": {"coordinates": [-87.631003848859, 41.8834456872408], "type": "Point"}}, {"properties": {"range": 3600, "fill": "#e5f5f9"}, "type": "Feature", "geometry": {"coordinates": [[[-88.2408142, 41.8139648], [-88.1982422, 41.8139648], [-88.1900024, 41.8167114], [-88.1845093, 41.8222046], [-88.1762695, 41.8249512], [-88.1652832, 41.8249512], [-88.1542969, 41.8276978], [-88.1433105, 41.8331909], [-88.1323242, 41.8359375], [-88.1213379, 41.8359375], [-88.1130981, 41.8386841], [-88.1103516, 41.8469238], [-88.1103516, 41.8579102], [-88.107605, 41.8661499], [-88.0911255, 41.8716431], [-88.0801392, 41.8826294], [-88.0801392, 41.8881226], [-88.0856323, 41.8936157], [-88.0883789, 41.9018555], [-88.0856323, 41.9100952], [-88.0691528, 41.9155884], [-88.0691528, 41.9210815], [-88.074646, 41.9265747], [-88.0801392, 41.9430542], [-88.0856323, 41.9485474], [-88.0883789, 41.9567871], [-88.0856323, 41.9650269], [-88.06